Mobilenet_v3_small

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler, random_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import time
import copy
import os
from collections import Counter

# Set device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Augmentation and Normalization for Training and Validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomRotation(20),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/MyDrive/clothing_data'  # Folder containing the dataset with subfolders for each class

# Load the entire dataset from the folder structure
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# Split dataset into training (80%) and validation (20%)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply validation transforms to the validation dataset
val_dataset.dataset.transform = data_transforms['val']

# Calculate Class Weights for Balanced Sampling on Training Set
train_labels = [full_dataset.targets[i] for i in train_dataset.indices]
class_counts = Counter(train_labels)
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
sample_weights = [class_weights[train_labels[i]] for i in range(len(train_labels))]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Create DataLoader with Balanced Sampling for Training
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)
}

dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
class_names = full_dataset.classes
num_classes = len(class_names)

# Load Pretrained Model (MobileNetV3 Small)
model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)  # Modify last layer for new num_classes
model = model.to(device)

# Loss Function, Optimizer, and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Early Stopping Parameters
early_stopping_patience = 10
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
early_stop_counter = 0

# Training Function
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    global best_acc, best_model_wts, early_stop_counter
    since = time.time()

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + Optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if validation accuracy improves
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    early_stop_counter = 0  # Reset early stopping counter
                else:
                    early_stop_counter += 1

        print()

        # Check early stopping condition
        if early_stop_counter >= early_stopping_patience:
            print("Early stopping triggered. Stopping training...")
            break

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the Model
model = train_model(model, criterion, optimizer, scheduler, num_epochs=50)

# Save the best model
torch.save(model.state_dict(), 'best_model_mobilenetv3.pth')

# Evaluation Function
def evaluate_model(model):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

# Evaluate the Model
evaluate_model(model)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/50
----------
train Loss: 1.1618 Acc: 0.5487
val Loss: 1.1626 Acc: 0.5617

Epoch 2/50
----------
train Loss: 0.4389 Acc: 0.8439
val Loss: 1.0289 Acc: 0.6605

Epoch 3/50
----------
train Loss: 0.3302 Acc: 0.8733
val Loss: 1.0047 Acc: 0.7037

Epoch 4/50
----------
train Loss: 0.1933 Acc: 0.9382
val Loss: 1.5582 Acc: 0.6235

Epoch 5/50
----------
train Loss: 0.1040 Acc: 0.9675
val Loss: 1.1610 Acc: 0.7099

Epoch 6/50
----------
train Loss: 0.1019 Acc: 0.9675
val Loss: 1.1284 Acc: 0.7099

Epoch 7/50
----------
train Loss: 0.1450 Acc: 0.9598
val Loss: 1.8917 Acc: 0.6358

Epoch 8/50
----------
train Loss: 0.0948 Acc: 0.9722
val Loss: 1.3727 Acc: 0.6790

Epoch 9/50
----------
train Loss: 0.0784 Acc: 0.9737
val Loss: 1.0431 Acc: 0.7469

Epoch 10/50
----------
train Loss: 0.0420 Acc: 0.9876
val Loss: 0.8558 Acc: 0.7778

Epoch 11/50
----------
train Loss: 0.0302 Acc: 0.9923
val Loss: 0.7587 Acc: 0.7778

Epoch 12/50
----------
train Loss: 0.0311 Acc: 0.9938
val Loss: 0.7088 Acc: 0.7963

E

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler, random_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import time
import copy
import matplotlib.pyplot as plt
from collections import Counter

# Set device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Augmentation and Normalization for Training and Validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomRotation(20),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/MyDrive/clothing_data'  # Folder containing the dataset with subfolders for each class

# Load the entire dataset from the folder structure
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# Split dataset into training (80%) and validation (20%)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply validation transforms to the validation dataset
val_dataset.dataset.transform = data_transforms['val']

# Calculate Class Weights for Balanced Sampling on Training Set
train_labels = [full_dataset.targets[i] for i in train_dataset.indices]
class_counts = Counter(train_labels)
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
sample_weights = [class_weights[train_labels[i]] for i in range(len(train_labels))]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Create DataLoader with Balanced Sampling for Training
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)
}

dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
class_names = full_dataset.classes
num_classes = len(class_names)

# Load Pretrained ResNet50 Model
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modify last layer for num_classes
model = model.to(device)

# Loss Function, Optimizer, and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Early Stopping Parameters
early_stopping_patience = 10
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
early_stop_counter = 0

# Initialize variables to track accuracy and loss for plotting
train_acc_history = []
val_acc_history = []
train_loss_history = []
val_loss_history = []

# Training Function
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    global best_acc, best_model_wts, early_stop_counter
    since = time.time()

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + Optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save loss and accuracy for plotting
            if phase == 'train':
                train_acc_history.append(epoch_acc.item())
                train_loss_history.append(epoch_loss)
            else:
                val_acc_history.append(epoch_acc.item())
                val_loss_history.append(epoch_loss)

            # Deep copy the model if validation accuracy improves
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                early_stop_counter = 0  # Reset early stopping counter
            elif phase == 'val':
                early_stop_counter += 1

        print()

        # Check early stopping condition
        if early_stop_counter >= early_stopping_patience:
            print("Early stopping triggered. Stopping training...")
            break

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the Model
model = train_model(model, criterion, optimizer, scheduler, num_epochs=50)

# Save the best model
torch.save(model.state_dict(), 'best_model_resnet50.pth')

# Plot Training and Validation Curves
def plot_curves():
    epochs = len(train_acc_history)
    plt.figure(figsize=(10, 5))

    # Plot accuracy curves
    plt.subplot(1, 2, 1)
    plt.plot(range(epochs), train_acc_history, label='Training Accuracy')
    plt.plot(range(epochs), val_acc_history, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss curves
    plt.subplot(1, 2, 2)
    plt.plot(range(epochs), train_loss_history, label='Training Loss')
    plt.plot(range(epochs), val_loss_history, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Display accuracy and loss curves
plot_curves()

# Function to show images and predictions
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))  # Convert tensor image to numpy
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean  # De-normalize
    inp = np.clip(inp, 0, 1)  # Clip to valid range
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # Pause to update plots

# Evaluation Function with Image Display
def evaluate_model(model, num_images=6):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []
    images_shown = 0

    fig = plt.figure(figsize=(12, 6))

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Display a few images with predictions
            if images_shown < num_images:
                for i in range(inputs.size(0)):
                    if images_shown >= num_images:
                        break
                    ax = plt.subplot(num_images // 2, 2, images_shown + 1)
                    ax.axis('off')
                    imshow(inputs.cpu().data[i])
                    ax.set_title(f'Predicted: {class_names[preds[i]]}, True: {class_names[labels[i]]}')
                    images_shown += 1

    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

# Evaluate the Model and display some predictions
evaluate_model(model)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 1.06 MiB is free. Process 4776 has 14.74 GiB memory in use. Of the allocated memory 14.14 GiB is allocated by PyTorch, and 475.74 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)